You have to work on the files:
*  [Books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz)
*  [Book ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz)
*  [Users](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz)
*  [Goodbooks books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz)
*  [Goodbooks ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz)

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    To read those files, you need to use the `encoding = 'latin-1'` option.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com), otherwise no  answer will be given.

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
books = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz", sep=';', encoding = 'latin-1')
bRatings = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz", sep=';', encoding = 'latin-1')
users = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz", sep=';', encoding = 'latin-1')
goodBooks = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz", sep=',', encoding = 'latin-1')
gbRatings = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz", sep=',', encoding = 'latin-1')

In [ ]:
books.head()

In [ ]:
bRatings.head()

In [ ]:
users.head()

In [ ]:
goodBooks.head()

In [ ]:
gbRatings.head()

In [ ]:
goodBooks['original_publication_year']=goodBooks['original_publication_year'].astype(int)
goodBooks['isbn13']=goodBooks['isbn13'].astype(float).astype(int)

### 1. Normalize the location field of *Users* dataset, splitting into city, region, country.

In [ ]:
# sfruttiamo come è costruita la colonna Location, con separatore la virgola
# dividiamo dunque ogni riga in tre parti e assegnamo ciascuna ad una colonna
occ = users["Location"].str.split(",", n = 2, expand = True)
users["City"]= occ[0]
users["Region"]= occ[1]
users["Country"]= occ[2]
users

,User-ID,Location,Age,City,Region,Country
0,1,"nyc, new york, usa",NaN,nyc,new york,usa
1,2,"stockton, california, usa",18.0,stockton,california,usa
2,3,"moscow, yukon territory, russia",NaN,moscow,yukon territory,russia
3,4,"porto, v.n.gaia, portugal",17.0,porto,v.n.gaia,portugal
4,5,"farnborough, hants, united kingdom",NaN,farnborough,hants,united kingdom
...,...,...,...,...,...,...
278853,278854,"portland, oregon, usa",NaN,portland,oregon,usa
278854,278855,"tacoma, washington, united kingdom",50.0,tacoma,washington,united kingdom
278855,278856,"brampton, ontario, canada",NaN,brampton,ontario,canada
278856,278857,"knoxville, tennessee, usa",NaN,knoxville,tennessee,usa


In [ ]:
# facciamo una valutazione dell'errore che abbiamo commesso o introdotto:
# rows rappresenta il numero di righe del dataframe users, per ogni riga cerchiamo le virgole
# e salviamo tutte le occorrenze in occ. Facciamo dunque un ciclo for per contare il numero di
# virgole contenuto nella singola riga e se diverso da 2 aumentiamo il contatore globale conta di 1
# stampiamo la percentuale di righe con un comportamento diverso dal previsto
rows = users.Location.size
conta = 0
for line in users.Location:
  occ = re.finditer(',',line)
  count = 0
  for match in occ:
    count +=1
  if count!=2:
    conta +=1
    #print(line)
    #print('-'*50)
print(f"percentuale comportamento diverso={round((conta/rows)*100,3)}%, numero={conta}, totale={rows}")

percentuale comportamento diverso=0.541%, numero=1510, totale=278858


In [ ]:
# siccome la percentuale di righe con un comportamento diverso è del 0.541% possiamo trascurarle
# consapevoli di aver introdotto dei potenziali errori. Potremmo in alternativa trovare soluzioni più accurate
# specifiche per trattare queste 1510 righe

### 2. For each book in the *Books* dataset, compute its average rating.

In [ ]:
# siccome le informazione relative al rating non sono contenute nel dataframe books, 
# facciamo il merge con il df bRatings sul campo ISBN che è comune ad entrambi i df 
joinBookRatings = pd.merge(books, bRatings, on='ISBN')
joinBookRatings.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,0


In [ ]:
# Identificato come unico un libro presi i campi ISBN, titolo, autore, anno di pubblicazione e publisher
# raggruppiamo per questi campi e calcoliamo la media del book-rating per ogni libro
# rinominiano in seguito la colonna per migliorare la comprensione e la visualizzazione
avgRatings = joinBookRatings.groupby(['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher'], as_index=False)['Book-Rating'].mean()[['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher','Book-Rating']]
avgRatings.rename({'Book-Rating': 'Media-Rating'}, axis=1, inplace=True)
avgRatings

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Media-Rating
0,0000913154,The Way Things Work: An Illustrated Encycloped...,C. van Amerongen (translator),1967,Simon &amp; Schuster,8.0
1,0001010565,Mog's Christmas,Judith Kerr,1992,Collins,0.0
2,0001046438,Liar,Stephen Fry,0,Harpercollins Uk,9.0
3,0001046713,Twopence to Cross the Mersey,Helen Forrester,1992,HarperCollins Publishers,0.0
4,000104687X,"T.S. Eliot Reading \The Wasteland\"" and Other ...",T.S. Eliot,1993,HarperCollins Publishers,6.0
...,...,...,...,...,...,...
270142,B000234N76,Falling Angels,Tracy Chevalier,2001,E P Dutton,0.0
270143,B000234NC6,It Must've Been Something I Ate: The Return of...,Jeffrey Steingarten,2002,Knopf,0.0
270144,B00029DGGO,"Good Wife Strikes Back, The",Elizabeth Buchan,0,Viking Adult,0.0
270145,B0002JV9PY,The Blockade Runners,Jules Verne,0,Digireads.com,0.0


In [ ]:
# Potenzialmente i valori sono sbagliati perché 0 potrebbe essere un valore di default in caso di elemento nullo
# e perché sensatamente quasi mai viene assegnata una valutazione così bassa per un numero così ampio di libri.
# Sarebbe meglio controllare la fonte dati oppure il alternativa considerare solamente i libri con un book-rating 
# diverso da zero prima di effettuare il calcolo della media-rating

In [ ]:
# mi copio joinBookRatings ed elimino tutte le righe che soddisfano la condizione Book-Rating nullo
# eseguo poi le stesse operazioni fatte in precedenza per ricavare la colonna media-rating
prova = joinBookRatings
prova = prova.drop(prova[prova['Book-Rating'] == 0].index)
avgRatings_2 = prova.groupby(['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher'], as_index=False)['Book-Rating'].mean()[['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher','Book-Rating']]
avgRatings_2.rename({'Book-Rating': 'Media-Rating'}, axis=1, inplace=True)
avgRatings_2

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Media-Rating
0,0000913154,The Way Things Work: An Illustrated Encycloped...,C. van Amerongen (translator),1967,Simon &amp; Schuster,8.0
1,0001046438,Liar,Stephen Fry,0,Harpercollins Uk,9.0
2,000104687X,"T.S. Eliot Reading \The Wasteland\"" and Other ...",T.S. Eliot,1993,HarperCollins Publishers,6.0
3,0001047213,The Fighting Man,Gerald Seymour,1993,HarperCollins Publishers,9.0
4,0001047973,Brave New World,Aldous Huxley,1999,Trafalgar Square Publishing,9.0
...,...,...,...,...,...,...
149828,B0001FZGPI,The Bonesetter's Daughter,Amy Tan,2001,Putnam Pub Group,7.0
149829,B0001FZGRQ,The Clan of the Cave Bear,Jean M. Auel,2001,Crown Publishing Group,9.0
149830,B0001GMSV2,Find Me,Rosie O'Donnell,2002,Warner Books,8.0
149831,B0001I1KOG,New York Public Library Literature Companion,New York Public Library,2001,Free Press,10.0


In [ ]:
# nel seguito utilizziamo avgRating ovvero considerando gli zeri come vere valutazioni

### 3. For each book in the *GoodBooks* dataset, compute its average rating.

In [ ]:
# ispezioniamo la colonna average_rating ordinandola in modo decrescente
goodBooks[['isbn', 'isbn13', 'authors', 'title', 'average_rating']].sort_values('average_rating',ascending=False)

,isbn,isbn13,authors,title,average_rating
24,545010225,9780545010220,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Deathly Hallows (Harry Po...,4.61
26,439785960,9780439785970,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Half-Blood Prince (Harry ...,4.54
23,439139600,9780439139600,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Goblet of Fire (Harry Pot...,4.53
17,043965548X,9780439655480,"J.K. Rowling, Mary GrandPrÃ©, Rufus Beck",Harry Potter and the Prisoner of Azkaban (Harr...,4.53
20,439358078,9780439358070,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Order of the Phoenix (Har...,4.46
...,...,...,...,...,...
33,1612130291,9781612130290,E.L. James,"Fifty Shades of Grey (Fifty Shades, #1)",3.67
27,140283331,9780140283330,William Golding,Lord of the Flies,3.64
2,316015849,9780316015840,Stephenie Meyer,"Twilight (Twilight, #1)",3.57
48,316160199,9780316160190,Stephenie Meyer,"New Moon (Twilight, #2)",3.52


In [ ]:
# Stabilito che un libro viene identificato univocamente da isbn, (isbn13), autore, titolo
# fatta la considerazione rating_n equivale a valutazione n, calcolo la media pesata e la ponga nella colonna rating_calcolato
# ritrovo un valore pari ad average_rating a meno di approssimazione alla seconda cifra decimale
# divido per work_ratings_count perché valore pari alla somma del numero dei vari rating_n
goodBooks['rating_calcolato'] = (goodBooks['ratings_1']*1+goodBooks['ratings_2']*2+goodBooks['ratings_3']*3+goodBooks['ratings_4']*4+goodBooks['ratings_5']*5)/goodBooks['work_ratings_count']
goodBooks[['isbn', 'isbn13', 'authors', 'title', 'average_rating','rating_calcolato']].sort_values('rating_calcolato',ascending=False)

,isbn,isbn13,authors,title,average_rating,rating_calcolato
24,545010225,9780545010220,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Deathly Hallows (Harry Po...,4.61,4.612756
26,439785960,9780439785970,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Half-Blood Prince (Harry ...,4.54,4.537725
17,043965548X,9780439655480,"J.K. Rowling, Mary GrandPrÃ©, Rufus Beck",Harry Potter and the Prisoner of Azkaban (Harr...,4.53,4.527866
23,439139600,9780439139600,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Goblet of Fire (Harry Pot...,4.53,4.525950
20,439358078,9780439358070,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Order of the Phoenix (Har...,4.46,4.463372
...,...,...,...,...,...,...
33,1612130291,9781612130290,E.L. James,"Fifty Shades of Grey (Fifty Shades, #1)",3.67,3.665074
27,140283331,9780140283330,William Golding,Lord of the Flies,3.64,3.642994
2,316015849,9780316015840,Stephenie Meyer,"Twilight (Twilight, #1)",3.57,3.571067
48,316160199,9780316160190,Stephenie Meyer,"New Moon (Twilight, #2)",3.52,3.521807


### 4. Merge together all rows sharing the same book title, author and publisher. We will call the resulting datset `merged books`. The books that have not been merged together will not appear in `merged books`.

In [ ]:
# raggruppiamo le righe di joinBookRatings che possiedono lo stesso titolo, autore e publisher e 
# contiamo il numero di occorrenza salvandolo nella colonna count. Successivamente, salviamo nel df
# merged_books solo i libri che sono stati merged ovvero quelli che hanno count >= 2
count = pd.DataFrame({'count': joinBookRatings.groupby( [ "Book-Title", "Book-Author", "Publisher"]).size()}).reset_index()
merged_books = count[count['count'] >1]

In [ ]:
merged_books

,Book-Title,Book-Author,Publisher,count
0,A Light in the Storm: The Civil War Diary of ...,Karen Hesse,Hyperion Books for Children,4
7,Deceived,Carla Simpson,Pinnacle Books,2
8,Earth Prayers From around the World: 365 Pray...,Elizabeth Roberts,HarperSanFrancisco,10
9,Final Fantasy Anthology: Official Strategy Gu...,David Cassady,BradyGames,4
10,Flight of Fancy: American Heiresses (Zebra Ba...,Tracy Cozzens,Kensington Publishing Corporation,2
...,...,...,...,...
264954,Ã?Ã?berraschung am Valentinstag.,Barbara Dobrick,"Fischer (Tb.), Frankfurt",2
264956,Ã?Ã?lpiraten.,Janwillem van de Wetering,Rowohlt Tb.,2
264957,Ã?Ã?rger mit Produkt X. Roman.,Joan Aiken,Diogenes Verlag,4
264959,Ã?Ã?stlich der Berge.,David Guterson,btb,3


### 5. For each book in `merged books` compute its average rating.

The average is computed considering all books in `books` that have been merged.

In [ ]:
# Per ogni libro di joinBookRatings, identificato da titolo, autore e publisher, dopo aver quindi raggruppato,
# calcoliamo la media di Book-rating e la poniamo nella colonna Media-rating del df merged_books
merged_books['Media-Rating'] = joinBookRatings.groupby(['Book-Title','Book-Author','Publisher'], as_index=False)['Book-Rating'].mean()['Book-Rating']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
merged_books

,Book-Title,Book-Author,Publisher,count,Media-Rating
0,A Light in the Storm: The Civil War Diary of ...,Karen Hesse,Hyperion Books for Children,4,2.250000
7,Deceived,Carla Simpson,Pinnacle Books,2,0.000000
8,Earth Prayers From around the World: 365 Pray...,Elizabeth Roberts,HarperSanFrancisco,10,5.000000
9,Final Fantasy Anthology: Official Strategy Gu...,David Cassady,BradyGames,4,5.000000
10,Flight of Fancy: American Heiresses (Zebra Ba...,Tracy Cozzens,Kensington Publishing Corporation,2,4.000000
...,...,...,...,...,...
264954,Ã?Ã?berraschung am Valentinstag.,Barbara Dobrick,"Fischer (Tb.), Frankfurt",2,0.000000
264956,Ã?Ã?lpiraten.,Janwillem van de Wetering,Rowohlt Tb.,2,0.000000
264957,Ã?Ã?rger mit Produkt X. Roman.,Joan Aiken,Diogenes Verlag,4,5.250000
264959,Ã?Ã?stlich der Berge.,David Guterson,btb,3,2.666667


In [ ]:
# ricordiamo anche in questo caso il discorso fatto in precedenza con i book-rating nulli

### 6. For each book in `merged books` compute the minimum and maximum of the average ratings over all corresponding books in the `books` dataset.

Hence for each book in `merged books` we will have exactly two values (a minimum and a maximum)

In [ ]:
# per ogni libro in merged_books, che corrisponde quindi a quelli di joinBookRatings, 
# identificato da titolo, autore, publisher, ISBN, calcoliamo la media
# salviamo il minimo e il massimo della media nelle due colonne corrispondenti di merged_books
df1 = pd.DataFrame({'Media-Rating':joinBookRatings.groupby(['Book-Title','Book-Author','Publisher','ISBN'])['Book-Rating'].mean()}).reset_index()
merged_books['min'] = df1.groupby(['Book-Title','Book-Author','Publisher'], as_index=False)['Media-Rating'].min()['Media-Rating']
merged_books['max'] = df1.groupby(['Book-Title','Book-Author','Publisher'], as_index=False)['Media-Rating'].max()['Media-Rating']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
merged_books

,Book-Title,Book-Author,Publisher,count,Media-Rating,min,max
0,A Light in the Storm: The Civil War Diary of ...,Karen Hesse,Hyperion Books for Children,4,2.250000,2.250000,2.250000
7,Deceived,Carla Simpson,Pinnacle Books,2,0.000000,0.000000,0.000000
8,Earth Prayers From around the World: 365 Pray...,Elizabeth Roberts,HarperSanFrancisco,10,5.000000,5.000000,5.000000
9,Final Fantasy Anthology: Official Strategy Gu...,David Cassady,BradyGames,4,5.000000,5.000000,5.000000
10,Flight of Fancy: American Heiresses (Zebra Ba...,Tracy Cozzens,Kensington Publishing Corporation,2,4.000000,4.000000,4.000000
...,...,...,...,...,...,...,...
264954,Ã?Ã?berraschung am Valentinstag.,Barbara Dobrick,"Fischer (Tb.), Frankfurt",2,0.000000,0.000000,0.000000
264956,Ã?Ã?lpiraten.,Janwillem van de Wetering,Rowohlt Tb.,2,0.000000,0.000000,0.000000
264957,Ã?Ã?rger mit Produkt X. Roman.,Joan Aiken,Diogenes Verlag,4,5.250000,5.250000,5.250000
264959,Ã?Ã?stlich der Berge.,David Guterson,btb,3,2.666667,2.666667,2.666667


### 7. For each book in `goodbooks`, compute the list of its authors. Assuming that the number of reviews with a text (column `work_text_reviews_count`) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the *shared number of reviews with a text*.

In [ ]:
# salviamo in un df tutte le righe di goodBooks con autori diversi, in modo da eliminare i duplicati
authors = goodBooks[goodBooks.duplicated(['authors'])==False]

In [ ]:
# definiamo una funzione che data una stringa elimini gli spazi in eccedenza 
def remove(string):
    return " ".join(string.split())

In [ ]:
# Creaimo un dizionario vuoto e decidiamo quali caratteri cercare (reg). Effettuiamo la precompilazione, 
# senza considerare differenze per minuscole e maiuscole. Per comodità in temp5 creaimo un dizionario
# composto da nome autore e work_text_reviews_count come valore
# Facciamo un loop dove cerchiamo tutte le occorrenze dei caratteri prescelti, ovvero dividiamo in vari autori. 
# Se troviamo almeno un'occorrenza (almeno un autore), facciamo un ciclo for per inserire il suo valore
# di work_text_reviews_count diviso per il numero di autori della riga con la seguente logica:
# se autore non ancora presente in myDict assegnare tutto il valore, altrimenti aggiornare il valore vecchio
# sommandogli la nuova work_text_reviews_count diviso per il numero di autori della riga
# utilizziamo la funzione remove per eliminare spazi in eccedenza e quindi non considerare due volte lo stesso autore
myDict= {}
reg = 'a-zA-Z.\s\-\u00A1-\u00FF'
re_enum = re.compile('(['+reg+']+),?', re.UNICODE)
temp5 = authors.set_index('authors')['work_text_reviews_count'].to_dict()
for key,value in temp5.items():
  occ = re_enum.findall(key)
  if occ:
    for j in range(len(occ)):
      if remove(occ[j]) not in myDict:
        myDict[remove(occ[j])] = int(value/len(occ))
      else:
        myDict[remove(occ[j])] = myDict[remove(occ[j])] + int(value/len(occ))

In [ ]:
# salviamo i risultati di myDict in un dataframe con le colonne autori e shared_number_of_reviews_with_text
shared_number_of_reviews_with_text = pd.DataFrame(myDict.items(), columns=['Autori', 'shared_number_of_reviews_with_text'])

In [ ]:
shared_number_of_reviews_with_text

In [ ]:
#ATTENZIONE TUTTO IL CODICE SOPRA FUNZIONA MA 
#RIGA 'George Orwell, Erich Fromm, CelÃ¢l Ãster'
#viene divisa male in 
#['George Orwell', ' Erich Fromm', ' CelÃ¢l Ã', 'ster']

reg = 'a-zA-Z.\s\-\u00A1-\u00FF'
re_enum = re.compile('(['+reg+']+),?', re.UNICODE)
occ = re_enum.findall('George Orwell, Erich Fromm, CelÃ¢l Ãster')
print(occ)

['George Orwell', ' Erich Fromm', ' CelÃ¢l Ã', 'ster']


### 8. For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.

In [ ]:
# Creaimo un dizionario vuoto e decidiamo quali caratteri cercare (reg). Effettuiamo la precompilazione, 
# senza considerare differenze per minuscole e maiuscole. Per comodità in temp6 creaimo un dizionario
# composto da nome autore e original_publication_year come valore
# Facciamo un loop dove cerchiamo tutte le occorrenze dei caratteri prescelti, ovvero dividiamo in vari autori. 
# Se troviamo almeno un'occorrenza (almeno un autore), facciamo un ciclo for per inserire in myDict2 con chiave
# original_publication_year il nome dell'autore con la seguente logica:
# se l'anno non è ancora presente in myDict2 creo un nuovo record e vi assegno il corrispondente autore
# se invece l'anno di pubblicazione è già presente in myDict2 recupero gli autori e vi aggiungo i nomi
# di quelli nuovi, separandoli con un virgola ognuno 
# utilizziamo la funzione remove per eliminare spazi in eccedenza 
myDict2= {}
reg = 'a-zA-Z.\s\-\u00A1-\u00FF'
re_enum = re.compile('(['+reg+']+),?', re.UNICODE)
temp6 = authors.set_index('authors')['original_publication_year'].to_dict()
for key,value in temp6.items():
  occ = re_enum.findall(key)
  if occ:
    j=0
    for j in range(len(occ)):
      if value not in myDict2:
        myDict2[value] = remove(occ[j])
      else:
        myDict2[value] = myDict2[value]+','+remove(occ[j])

In [ ]:
# Creaimo un dizionario vuoto myDict3. Effettuiamo uno doppio ciclo su myDict2 contenente
# autori e anni di pubblicazione e su myDict contenente autori e numero di review. 
# Se troviamo l'autore in myDict, se l'anno associato non è presente ancora in myDict3 come chiave
# allora creiamo un record con chiave anno e vi associamo il dizionario composto da 
# {'autore':nome autore,'review':numero di reviews}
# Nel caso in cui l'anno fosse invece già presente confrontiamo il numero di review e 
# se maggiore sostituiamo nome autore e numero di review 
# Generiamo poi un dataframe prima con chiave anno, poi con indice esterno
myDict3 = {}
for anno,aut in myDict2.items():
  for autore,review in myDict.items():
    if re.search(autore,aut):
      if anno not in myDict3:
        myDict3[anno] = {'autore':autore,'review':review}
      else:
        if myDict3[anno]['review'] <= review:
          myDict3[anno]['review'] = review
          myDict3[anno]['autore'] = autore
year_reviews =pd.DataFrame.from_dict(myDict3,orient='index')
year_reviews.index.name = 'anno'
year_reviews.reset_index(level=['anno'])

### 9. Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets

In [ ]:
# Assumiamo che non vi siano errori nel campo ISBN. Ci salviamo nel dataframe libri i libri di joinBookRating
# dopo averli opportunamente raggruppati per ISBN per calcolare la media di Book-rating
# Analogo procedimento con good_libri per i libri in goodBooks
# Nella lista isbn_comuni mi salvo tutti i codici ISBN comuni nei due dataframe
libri = pd.DataFrame({'Rating_books':joinBookRatings.groupby(["ISBN"])['Book-Rating'].mean()}).reset_index()
good_libri = pd.DataFrame({'Rating_books':goodBooks.groupby(['isbn',"isbn13"])['average_rating'].mean()}).reset_index()
isbn_comuni = good_libri[good_libri.isbn.isin(libri.ISBN)]['isbn'].to_list()

In [ ]:
# genero un array vuoto row_list. Scorrendo con un for sugli ISBN in isbn_comuni
# mi salvo in val1 il valore di Rating-books recuperato da libri mentre in val2 il valore 
# recuperato da good_libri. Creo un dizionario dict1 che abbia come chiavi isbn, Rating_books,
# Rating_goodbooks e difference dove associo il corrispondente valore (differenza in valore assooluto 
# del valore di rating). Appendo poi il dizionario nell'array row_list. 
# Uscito dal ciclo genero un df con i dati raccolti in row_list
row_list = []
for lib in isbn_comuni:
  val1 = libri[libri['ISBN']== lib]['Rating_books'].values[0]
  val2 = good_libri[good_libri['isbn']==lib]['Rating_books'].values[0]
  dict1 = {
      'isbn'             : lib,
      'Rating_books'     : val1,
      'Rating_goodbooks' : val2,
      'Difference'       : abs(val2-val1)
  }
  row_list.append(dict1)
df4 = pd.DataFrame(row_list, columns=['isbn','Rating_books','Rating_goodbooks','Difference'])
df4

### 10. Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.

In [ ]:
# Prima di tutto cerco di capire la massima età che sia presente in users -> 244
users[users['Age'] == users['Age'].max()]

In [ ]:
# Divido il dataset users secondo blocchi di età. Prima creo il gruppo_0_14 separatamente
# Poi tramite un ciclo su i da 1 a 23 compreso genereo tutti gli altri con il nome che varia 
# secondo il tipo group_25_34 -> group 35_44 ecc. una volta generate tutte le variabili le aggiorno
# per il dizionario var_holder  
group_0_14 = users[users['Age']<15]
var_holder = {}
for i in range(1,24):
  var_holder["group_"+ str(i)+"5_"+str(i+1)+"4"] = users[(users['Age']>int(str(i)+"4")) & (users['Age']<int(str(i+1)+"5"))]
locals().update(var_holder)
#sono in dataframe con nome tipo group_25_34

### 11. Find the books that appear only in the goodbooks datasets.

In [ ]:
# cerco i libri che compaiono solo nel dataframe goodbooks. Per farlo uso il metodo
# isin che cerca i libri con isbn presente sia in libri che in good_libri 
# poi con ~ faccio la negazione. Salvo solo i libri di goodBooks che hanno un isbn tra questi
solo_in_goodbooks = goodBooks[~good_libri.isbn.isin(libri.ISBN)]

In [ ]:
solo_in_goodbooks

### 12. Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?

In [ ]:
# Assumendo che la coppia autore, titolo identifichi un libro, per ogni libro cerchiamo il numero di volte che 
# compare nel df books. Generiamo un df count_book che è composto dal raggruppamento di titolo e autore dei libri
# in books, dove calcoliamo la dimensione del raggruppamento (ovvero il numero di occorrenze) e lo salviamo nella
# colonna count. Per trovare il libro che compare il maggior numero di volte cerchiamo la riga che ha
# valore di count massimo -> 21
count_book = pd.DataFrame({'count': books.groupby( [ "Book-Title", "Book-Author"] ).size()}).reset_index()
count_book[count_book['count'] == count_book['count'].max()]

In [ ]:
count_book.sort_values('count',ascending=False)

### 13. Find the author with the highest average rating according to the goodbooks datasets.

In [ ]:
'''
esempio funzionamento
Pino,Carlo 10
Pino,Giulio 8
Pino,Fede 0

Pino    rating: 10 num_occ: 1
Carlo   rating: 10 num_occ: 1

Pino    rating: 18 num_occ: 2
Carlo   rating: 10 num_occ: 1
Giulio  rating: 8  num_occ: 1

Pino    rating: 18 num_occ: 3
Carlo   rating: 10 num_occ: 1
Giulio  rating: 8  num_occ: 1
Fede    rating: 0  num_occ: 1
'''

In [ ]:
# Creaimo un dizionario vuoto myDict5 e decidiamo quali caratteri cercare (reg). Effettuiamo la precompilazione, 
# senza considerare differenze per minuscole e maiuscole. Per comodità in temp8 creaimo un dizionario
# composto da nome autore e average-rating come valore
# Facciamo un loop dove cerchiamo tutte le occorrenze dei caratteri prescelti, ovvero dividiamo la riga in vari autori. 
# Se troviamo almeno un'occorrenza (almeno un autore), facciamo un ciclo for con la seguente logica:
# se autore non ancora presente in myDict5 assegnare un dizionario con il valore di rating e numero di occ ovvero
# il numero di autori con cui divide tale rating, altrimenti aggiornare il valore vecchio di rating sommandovi il nuovo
# utilizziamo la funzione remove per eliminare spazi in eccedenza e quindi non considerare due volte lo stesso autore come chiave
myDict5= {}
reg = 'a-zA-Z.\s\-\u00A1-\u00FF'
re_enum = re.compile('(['+reg+']+),?', re.UNICODE)
temp8 = authors.set_index('authors')['average_rating'].to_dict()
for key,value in temp8.items():
  occ = re_enum.findall(key)
  if occ:
    for j in range(len(occ)):
      if remove(occ[j]) not in myDict5:
        #myDict5[remove(occ[j])] = {'rating':value,'num_occ':len(occ)}
        myDict5[remove(occ[j])] = {'rating':value,'num_occ':1}
      else:
        #myDict5[remove(occ[j])] = {'rating':myDict5[remove(occ[j])]['rating']+value,'num_occ':len(occ)}
        myDict5[remove(occ[j])] = {'rating':myDict5[remove(occ[j])]['rating']+value,'num_occ':myDict5[remove(occ[j])]['num_occ']+1}

In [ ]:
# creaiamo un df con i valori presi da myDict5 (avremo due colonne Autori e 
# rating_plus_count fatta così {'rating': 4.34, 'num_occ': 1})
author_max_rating = pd.DataFrame(myDict5.items(), columns=['Autori', 'rating_plus_count'])
# normalizziamo la colonna rating_plus_count divendola in due diverse colonne e mettiamo il tutto nel df normalized
normalized = pd.json_normalize(author_max_rating['rating_plus_count'])
# ora fondiamo il df author_max_rating e normalized droppando la vecchia colonna rating_plus_count 
author_max_rating = author_max_rating.join(normalized).drop(columns=['rating_plus_count'])
# ricalcoliamo la colonna rating facendo il rapporto tra valore di rating e numero di occorrenze
author_max_rating['rating']= author_max_rating['rating']/author_max_rating['num_occ']
# possiamo ora droppare la colonna num_occ che non ci serve più
author_max_rating.drop('num_occ', axis=1, inplace=True)
author_max_rating

In [ ]:
# cerchiamo l'indice dell'autore con valore massimo di rating in modo da potere mostrare tutto il record
author_max_rating.loc[author_max_rating['rating'].idxmax()]

Autori    Rufus Beck
rating          4.53
Name: 22, dtype: object